# Construct a Bayesian interval 

using the [RooStats::BayesianCalculator](hhttps://root.cern.ch/doc/v608/classRooStats_1_1BayesianCalculator.html) from the model and data stored in [model.root](ex02_build_Poisson.ipynb)

In [1]:
%jsroot

Open the ROOT file

In [2]:
TFile* f = TFile::Open("../workspaces/model.root") ;


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Retrieve the workspace

In [3]:
RooWorkspace* w = (RooWorkspace*) f->Get("w") ;
w->Print() ;


RooWorkspace(w) w contents

variables
---------
(B,Nobs,S,mu)

p.d.f.s
-------
RooPoisson::model[ x=Nobs mean=Nexp ] = 0.0511153

functions
--------
RooFormulaVar::Nexp[ actualVars=(mu,S,B) formula="mu*S+B" ] = 30

datasets
--------
RooDataSet::observed_data(Nobs)

parameter snapshots
-------------------
ModelConfig__snapshot = (mu=1)

named sets
----------
ModelConfig_Observables:(Nobs)
ModelConfig_POI:(mu)
ModelConfig__snapshot:(mu)

generic objects
---------------
RooStats::ModelConfig::ModelConfig



Retrieve the ModelConfig and the observed data. Together these uniquely define the statistical problem

In [4]:
RooAbsData* data = w->data("observed_data") ;
RooStats::ModelConfig* mc = (RooStats::ModelConfig*) w->obj("ModelConfig") ;

Instantiate a Bayesian interval calculator

In [5]:
RooStats::BayesianCalculator bayesianCalc(*data,*mc);

## Statistical configuration of Bayesian interval calculation

Calculate the 90% C.L. central interval

In [6]:
bayesianCalc.SetConfidenceLevel(0.90);

Choose central interval

In [7]:
bayesianCalc.SetLeftSideTailFraction(0.5);
//bayesianCalc.SetLeftSideTailFraction(0);  // alternative for upper limit
//bayesianCalc.SetShortestInterval();       // alternative for shortest intervals

Optionally introduce a prior pdf on $\mu$

In [8]:
bool use_prior=false ;
if (use_prior) {
    w->factory("EXPR::prior('1/sqrt(mu)',mu)") ;
    bayesianCalc.SetPriorPdf(*w->pdf("prior")); 
}

## Technical parameter of Bayesian calculator

Choose best numeric integration algorithm

In [9]:
RooAbsReal::defaultIntegratorConfig()->method1D().setLabel("RooAdaptiveGaussKronrodIntegrator1D");

Select number of scan points

In [10]:
bayesianCalc.SetScanOfPosterior(500);

Calculate interval

In [11]:
RooStats::SimpleInterval* interval = bayesianCalc.GetInterval();

[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_model_FOR_OBS_Nobs with 0 entries
[#1] INFO:Eval -- BayesianCalculator::GetPosteriorFunction :  nll value 2.97367 poi value = 1
[#1] INFO:Eval -- BayesianCalculator::GetPosteriorFunction : minimum of NLL vs POI for POI =  0.500023 min NLL = 2.53171
[#1] INFO:Minization -- createNLL picked up cached consraints from workspace with 0 entries
[#1] INFO:Eval -- BayesianCalculator - scan posterior function in nbins = 500
[#1] INFO:Eval -- BayesianCalculator::GetInterval - found a valid interval : [-0.178229 , 1.49173 ]


Print the result

In [12]:
RooRealVar* poi = (RooRealVar*) mc->GetParametersOfInterest()->first();
double lowerLimit = interval->LowerLimit();
double upperLimit = interval->UpperLimit();
cout << "RESULT: " << 100*bayesianCalc.ConfidenceLevel() << "% interval is : ["<< lowerLimit << ", "<< upperLimit <<"] "<<endl;

RESULT: 90% interval is : [-0.178229, 1.49173] 


Use the visualization tool of the Bayesian calculator to show how the interval was calculated

In [13]:
TCanvas* c = new TCanvas();
RooPlot* plot = bayesianCalc.GetPosteriorPlot();
plot->Draw() ; 
c->Draw();